In [ ]:
from keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [ ]:
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))

In [ ]:
from matplotlib import pyplot
for i in range(9):  
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(train_X[i], cmap=pyplot.get_cmap('gray'))
pyplot.show()

In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


# --- SUBSAMPLE for speed ---
n_train = 2000
n_test = 500

train_X = train_X[:n_train]
train_y = train_y[:n_train]
test_X = test_X[:n_test]
test_y = test_y[:n_test]

# Convert grayscale 28x28 → RGB 224x224x3
train_X_rgb = np.repeat(train_X[..., np.newaxis], 3, -1)
test_X_rgb = np.repeat(test_X[..., np.newaxis], 3, -1)
train_X_resized = tf.image.resize(train_X_rgb, (224, 224)).numpy()
test_X_resized = tf.image.resize(test_X_rgb, (224, 224)).numpy()

# Preprocess for ResNet
train_X_resized = preprocess_input(train_X_resized)
test_X_resized = preprocess_input(test_X_resized)

# Load pretrained ResNet (without top classifier)
resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract embeddings
train_embeddings = resnet.predict(train_X_resized, verbose=1)
test_embeddings = resnet.predict(test_X_resized, verbose=1)

In [ ]:
# KNN classifier
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn.fit(train_embeddings, train_y)

y_pred = knn.predict(test_embeddings)
acc = accuracy_score(test_y, y_pred)
print(f"KNN on pretrained ResNet embeddings (n_train={n_train}, n_test={n_test}): {acc:.4f}")

# Precision, Recall, F1 report
report = classification_report(test_y, y_pred, digits=3)
print("\nClassification Report:")
print(report)

# Confusion matrix
cm = confusion_matrix(test_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap='Blues', values_format='d')
plt.title(f"Confusion Matrix – KNN on ResNet Embeddings ({n_train} train, {n_test} test)")
plt.show()


In [ ]:
from sklearn.svm import SVC

# SVM classifier
svm = SVC(kernel='rbf', C=1.0, gamma='scale')  # You can tweak C and kernel
svm.fit(train_embeddings, train_y)

y_pred = svm.predict(test_embeddings)
acc = accuracy_score(test_y, y_pred)
print(f"SVM on pretrained ResNet embeddings (n_train={n_train}, n_test={n_test}): {acc:.4f}")

# Precision, Recall, F1 report
report = classification_report(test_y, y_pred, digits=3)
print("\nClassification Report:")
print(report)

# Confusion matrix
cm = confusion_matrix(test_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap='Blues', values_format='d')
plt.title(f"Confusion Matrix – KNN on ResNet Embeddings ({n_train} train, {n_test} test)")
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Convert labels to one-hot
train_y_cat = to_categorical(train_y, 10)
test_y_cat = to_categorical(test_y, 10)

# Define a simple MLP
mlp = Sequential([
    Dense(256, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

# Compile
mlp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = mlp.fit(train_embeddings, train_y_cat,
                  validation_split=0.1,
                  epochs=10,
                  batch_size=64,
                  verbose='no')

# Evaluate
test_loss, test_acc = mlp.evaluate(test_embeddings, test_y_cat)
print(f"MLP on pretrained ResNet embeddings (n_train={n_train}, n_test={n_test}): {test_acc:.4f}")


# Predictions
y_pred = mlp.predict(test_embeddings).argmax(axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(test_y, y_pred, digits=3))

# Confusion matrix
cm = confusion_matrix(test_y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix – MLP on ResNet Embeddings")
plt.show()
